# Prospecção de Dados (Data Mining) DI/FCUL - HA2

## Course Project (MC/DI/FCUL - 2024)

### GROUP: `02`

* João Martins, 62532 - Hours worked on the project: 16
* Rúben Torres, 62531 - Hours worked on the project: 16
* Nuno Pereira, 56933 - Hours worked on the project: 16

### 1. Import the dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    confusion_matrix,
    accuracy_score,
    classification_report, 
    explained_variance_score, 
    mean_squared_error, 
    max_error, 
    mean_absolute_error
)

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from scipy.stats import pearsonr

with open("mol_bits.pkl", "rb") as file:
    molecular_fingerprints = pickle.load(file)

# Convert molecular fingerprints to DataFrame
mol_df = pd.DataFrame(
    list(molecular_fingerprints.items()), columns=["Molecules", "Fingerprint"]
)
mol_df

In [ ]:
activity = pd.read_csv(
    "activity_train.csv", header=None, names=["Proteins", "Molecules", "Rate"]
)
activity

In [ ]:
activity_test = pd.read_csv(
    "activity_test_blanked.csv", header=None, names=["Proteins", "Molecules", "Rate"]
)
activity_test

In [ ]:
# Count the number of times each protein appears
protein_counts = activity['Proteins'].value_counts()

# Sort proteins by their counts in descending order
sorted_proteins = protein_counts.index.tolist()[:30]
counts = protein_counts.values[:30]

# Plot the bar chart
plt.figure(figsize=(7, 4))
plt.bar(range(len(sorted_proteins)), counts, color='skyblue')
plt.xlabel('Proteins')
plt.ylabel('Number of Molecules')
plt.title('Number of Molecules in Each Protein')
plt.xticks(range(len(sorted_proteins)), sorted_proteins, rotation=60)
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of unique proteins each molecule appears in
molecule_protein_counts = activity.groupby('Molecules')['Proteins'].nunique()

# Sort molecules by the number of unique proteins they appear in, in descending order
sorted_molecules = molecule_protein_counts.sort_values(ascending=False).index.tolist()[:50]
counts = molecule_protein_counts.sort_values(ascending=False).values[:50]

# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.bar(range(len(sorted_molecules)), counts, color='skyblue')
plt.xlabel('Molecules')
plt.ylabel('Number of Proteins')
plt.title('Number of Proteins Each Molecule Appears In')
plt.xticks(range(len(sorted_molecules)), sorted_molecules, rotation=70)
plt.tight_layout()
plt.show()

In [ ]:
molecule_protein_counts = mol_df.set_index('Molecules')['Fingerprint'].apply(len)

# Sort molecules by the number of proteins they appear in, in descending order
sorted_molecules = molecule_protein_counts.sort_values(ascending=False).index.tolist()[:50]
counts = molecule_protein_counts.sort_values(ascending=False).values[:50]

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(range(len(sorted_molecules)), counts, color='skyblue')
plt.xlabel('Molecules')
plt.ylabel('Finger print length')
plt.title('Number of Proteins Each Molecule Appears In')
plt.xticks(range(len(sorted_molecules)), sorted_molecules, rotation=90)
plt.tight_layout()
plt.show()

* The file activity_train.csv contains a list of interactions between molecules (identified by their ChEMBL IDs and proteins identified by their Uniprot IDs). The activity value is rated from 1 to 10, where 1 is INACTIVE and 10 is EXTREMELY POTENT.

* The file activity_test_blanked.csv has exactly the same structure as activity_train.csv, yet, the activiy values are all at Zero. The goal of the project is to predict the real values.

* Additionally it is provided the Fingerprints of molecules (mol_bits.pkl). Fingerprinting is a hashed structural representation of molecules, where each set bit represents a structural feature. Molecules that have a common bit set mean that they probably share a structural element. This file is a Zipped pickled file that contain a dictionary with keys corresponding the ChEMBL IDs and values corresponding to a list of the set bits of each molecule.

### 2. Merge the dataset

In [ ]:
def convert_to_bit_vector(bit_indices, length):
    bit_vector = np.zeros(length, dtype=int)
    bit_vector[bit_indices] = 1
    return bit_vector


max_bit_index = max([max(fp) for fp in molecular_fingerprints.values()])

bit_vectors = mol_df["Fingerprint"].apply(
    lambda x: convert_to_bit_vector(x, max_bit_index + 1)
)
bit_matrix = np.vstack(bit_vectors)
bit_df = pd.DataFrame(bit_matrix, columns=[f"{i}" for i in range(max_bit_index + 1)])
mol_df = pd.concat([mol_df[["Molecules"]], bit_df], axis=1)


activity["Molecules"] = activity["Molecules"].str.strip()
mol_df["Molecules"] = mol_df["Molecules"].str.strip()
activity_test["Molecules"] = activity_test["Molecules"].str.strip()

In [ ]:
merged_df = pd.merge(activity, mol_df, on="Molecules")
merged_df

merged_df_test = pd.merge(activity_test, mol_df, on="Molecules")
merged_df

In [ ]:
# Prepare features (X) and target variable (y)
X = merged_df.drop(columns=["Proteins", "Molecules", "Rate"])
y = merged_df["Rate"]

X_test_preds = merged_df_test.drop(columns=["Proteins", "Molecules", "Rate"])


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### 3. PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1150)
pca.fit(X)
tve=0
for i, ve in enumerate(pca.explained_variance_ratio_):
    tve+=ve
    print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve) )
print()
print("Actual Eigenvalues:", pca.singular_values_)
for i,comp in enumerate(pca.components_):
    print("PC",i, "-->", comp)

In [ ]:
X_train_Sscaled_PCA = pca.transform(X_train)
X_test_Sscaled_PCA = pca.transform(X_test)
print(X_train_Sscaled_PCA.shape)

### 4. ML model

In [ ]:
def printRegStatistics(truth, preds):
    print("The RVE is: ", explained_variance_score(truth, preds))
    print("The rmse is: ", mean_squared_error(truth, preds, squared=False))
    corr, pval = pearsonr(truth, preds)
    print("The Correlation Score is is: %6.4f (p-value=%e)\n"%(corr,pval))
    print("The Maximum Error is is: ", max_error(truth, preds))
    print("The Mean Absolute Error is: ", mean_absolute_error(truth, preds))

def printClassResults(model_name, truth, preds):
    print("The Model is: %s" % model_name)
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds, average="weighted"))
    print("The Recall is: %7.4f" % recall_score(truth, preds, average="weighted"))
    print("The F1 score is: %7.4f" % f1_score(truth, preds, average="weighted"))
    print(
        "The Matthews correlation coefficient is: %7.4f"
        % matthews_corrcoef(truth, preds)
    )
    print()
    print("The Confusion Matrix is:")
    print(pd.DataFrame(confusion_matrix(truth, preds)))
    print("\n")

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

y_pred1 = reg.predict(X_test)

printRegStatistics(y_test, y_pred1)
y_pred1.shape

In [ ]:
log = LogisticRegression().fit(X_train, y_train)

y_pred3= log.predict(X_test)

accuracy = accuracy_score(y_test, y_pred3)
printClassResults("LogisticRegression", y_test, y_pred3)


In [ ]:
Nb_classifier = GaussianNB()
Nb_classifier.fit(X_train, y_train)

In [ ]:
y_pred2 = Nb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred2)
printClassResults("GaussianNB", y_test, y_pred2)


In [ ]:
rf_classifier = RandomForestClassifier(max_depth=40, n_jobs=-1)
rf_classifier.fit(X, y)

In [ ]:
y_pred = rf_classifier.predict(X_test_preds)

# accuracy = accuracy_score(y_test, y_pred)
# printClassResults("RandomForestClassifier", y_test, y_pred)


In [ ]:
activity_test["Rate"] = y_pred

activity_test.to_csv("PD_PREDS-02.csv", index=False)
activity_test